In [59]:
import numpy as np

In [49]:
#XOR gate with one hidden layer
class XOR_gate:
    def __init__(self):
        self.Hlayer = 2
        self.inputs = 2
        self.out = 1
        self.w1  = np.random.randn(self.inputs+1,self.Hlayer)
        self.w2 = np.random.randn(self.Hlayer+1,self.out)
    
    def sigmoid(slef,z):
        return 1/(1+np.exp(-z))
  
    def sigmoidderiv(self,z):
        return self.sigmoid(z)*(1 - self.sigmoid(z))
  
    def forward(self,X):
        self.X = np.hstack((X,np.ones((X.shape[0],1)))) 
        self.a1 = np.matmul(self.X,self.w1)
        self.z1 = self.sigmoid(self.a1)
        self.z2 = np.hstack((self.z1,np.ones((self.z1.shape[0],1))))
        self.a2 = np.matmul(self.z2,self.w2)
        self.z3 = self.sigmoid(self.a2)
        return self.z3
    
    def costfunction(self,X,y):
        self.loss = sum((self.forward(X)-y)**2)/2
        return self.loss
    
    def costfunctionderiv(self,X,y):
        self.y_hat = self.forward(X)
        self.delta2 = (self.y_hat-y)*self.sigmoidderiv(self.a2)
        self.dCdW2 = np.matmul(self.z2.T,self.delta2)
        
        self.delta1 = np.matmul(((self.delta2 * self.X).T),self.sigmoidderiv(self.a1))
        self.dCdW1  = self.w2[0:-1,:].T * self.delta1
        return self.dCdW1,self.dCdW2
    
    const = 1
    
    def train(self,X,y,num=100):
        if num==0: return
        self.rw1,self.rw2 = self.costfunctionderiv(X,y)
        self.w1 -= self.rw1*self.const
        self.w2 -= self.rw2*self.const
        self.train(X,y,num-1)        
        
    def self_train(self):
        self.X1 = np.array([[0,0],[0,1],[1,1],[1,0]])
        self.y = np.array([[0],[1],[0],[1]])
        for i in range(100):
            self.train(self.X1,self.y)
        
    def output(self,X):
        return np.round(self.forward(X))     
        
         
        

In [50]:

XORgate = XOR_gate()
X= np.array([[0,1],[1,1],[0,0],[1,0]])
y = np.array([[1],[0],[0],[1]])
for i in range(1000):
    XORgate.train(X,y)
    

In [51]:
#test it out
print(XORgate.forward(X))
print(XORgate.output(np.array([[1,1]])))

#XOR gate works with one hidden layer implemented


[[0.996734  ]
 [0.00309873]
 [0.00343399]
 [0.99603421]]
[[0.]]


In [52]:
#full adder
#creating class gate for AND and OR gates
class gate:
  def __init__ (self):
       self.Weights = np.random.randn(3,1)
  
  
  def sigmoid(slef,z):
    return 1/(1+np.exp(-z))
  
  def sigmoidderiv(self,z):
    return self.sigmoid(z)*(1 - self.sigmoid(z))
  
  def forward(self,X):
    X = np.hstack((X, np.ones((X.shape[0],1))))   
    self.a = np.matmul(X,self.Weights)
    self.z = self.sigmoid(self.a)
    return self.z
  
  
  def costfunction(self,X,y):
    self.loss = sum((self.forward(X)-y)**2) /2
    return self.loss
  
  
  def costfunctionderiv(self,X,y):
    self.y_hat = self.forward(X)
    X = np.hstack((X, np.ones((X.shape[0],1))))  
    self.delta = (self.y_hat - y)*self.sigmoidderiv(self.a)
    self.dCdW = np.matmul(X.T,self.delta)   
    return self.dCdW  
  
  def train(self,X,y,num):
    if num == 0: return
    const = 1
    self.Weights = self.Weights - const*self.costfunctionderiv(X,y) 
    self.train(X,y,num-1)

  def self_train_and(self):
    self.matrix = np.array([[0,0],[0,1],[1,0],[1,1]])
    self.y = np.array([[0],[0],[0],[1]])
    for i in range(100):
      self.train(self.matrix,self.y,1000)

  def self_train_or(self):
   self.matrix = np.array([[0,0],[0,1],[1,0],[1,1]])
   self.y = np.array([[0],[1],[1],[1]])
   for i in range(100):
      self.train(self.matrix,self.y,1000)


  def output(self,X):
    return np.round(self.forward(X))

In [53]:
class multi:
    gate_AND = gate()
    gate_AND.self_train_and()
    gate_OR = gate()
    gate_OR.self_train_or()
    gate_XOR = XOR_gate()
    gate_XOR.self_train()
    
    def OR(self,X):  
        self.rem = X.pop()
        for _ in range(len(X)):
            self.t = X.pop()
            self.rem = self.gate_OR.output(np.array([[self.rem,self.t]]))[0,0]
        return self.rem
    def AND(self,X):
        self.rem = X.pop()
        for _ in range(len(X)):
            self.t = X.pop()
            self.rem = self.gate_AND.output(np.array([[self.rem,self.t]]))[0,0]
        return self.rem 
    
    def XOR(self,X):
        self.rem = X.pop()
        for _ in range(len(X)):
            self.t = X.pop()
            self.rem = self.gate_XOR.output(np.array([[self.rem,self.t]]))[0,0]
        return self.rem
    
g1 = multi()
        
        

In [54]:
def full_adder(A,B,C_in):
    input = [A,B,C_in]
    Sum = g1.XOR(input)
    C_out = g1.OR([g1.AND([A,B]),g1.AND([A,C_in]),g1.AND([B,C_in])])
    return Sum, C_out
    

In [55]:
#Sum,C_out = full_adder(1,1,0)
#print(Sum,C_out)

In [56]:
def ripple_carry_adder(A,B):
    C_in = 0 #default carry-in
    ans = 0
    while (A or B):
        s,C_out = full_adder(A%10,B%10,C_in)
        ans = 10*ans + s
        C_in = C_out
        A = A//10
        B = B//10
    ans = ans*10 + C_in
    out = 0
    while(ans):
        out = out*10 + ans%10
        ans = ans//10 
    return out

In [58]:
print(ripple_carry_adder(11111001,11110))

100010111.0
